# Milestone 2

In [1]:
%cd ..
# imports 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scripts_m2 import *


c:\Users\001\anaconda3\envs\nlp_env\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\Users\001\OneDrive\Desktop\GUC\semester 10\nlp\NLP_PROJECT_111\QA Task milestone


### 1. Exploring the Dataset

In [2]:
# Loading the training data
import json 

with open('data/m2_train.json', 'r') as f:
    squad_data = json.load(f)

print(squad_data['data'][0]['paragraphs'][0].keys())

print(json.dumps(squad_data['data'][0], indent=2))

dict_keys(['qas', 'context'])
{
  "title": "Beyonc\u00e9",
  "paragraphs": [
    {
      "qas": [
        {
          "question": "When did Beyonce start becoming popular?",
          "id": "56be85543aeaaa14008c9063",
          "answers": [
            {
              "text": "in the late 1990s",
              "answer_start": 269
            }
          ],
          "is_impossible": false
        },
        {
          "question": "What areas did Beyonce compete in when she was growing up?",
          "id": "56be85543aeaaa14008c9065",
          "answers": [
            {
              "text": "singing and dancing",
              "answer_start": 207
            }
          ],
          "is_impossible": false
        },
        {
          "question": "When did Beyonce leave Destiny's Child and become a solo singer?",
          "id": "56be85543aeaaa14008c9066",
          "answers": [
            {
              "text": "2003",
              "answer_start": 526
            }
          ],


In [3]:
# Counting the number of answerable and unanswerable questions
num_answerable = 0
num_unanswerable = 0

for article in squad_data["data"]:
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            if qa["is_impossible"]:
                num_unanswerable += 1
            else:
                num_answerable += 1

print("Number of answerable questions: ", num_answerable)
print("Number of unanswerable questions: ", num_unanswerable)
print("Number of QA pairs: ", num_answerable + num_unanswerable)

Number of answerable questions:  86821
Number of unanswerable questions:  43498
Number of QA pairs:  130319


Answerable vs. Unanswerable Questions in SQuAD 2.0

- **Answerable Questions:** The correct answer exists within the given passage, and the model must extract the exact span.
- **Unanswerable Questions:** The passage does not contain the answer, and the model should predict "No Answer."

SQuAD 2.0 introduces unanswerable questions to make the task more challenging, requiring models to distinguish between when an answer is present and when it is not.

In this notebook, we will use answerable questions only in the subset we will be training our neural networks on

### 2. Preprocessing the dataset

In [4]:
# Clean text function
import re

def clean_text(text: str, is_question: bool=False):
    """
    Cleans text by removing extra spaces, newline characters, and special symbols.
    """
    if is_question:
        text = text.replace("\n", " ").replace("\t", " ")  # Remove newlines & tabs
        text = re.sub(r"\s+", " ", text)  # Remove extra spaces
        text = text.strip()  # Trim leading/trailing spaces
    text = text.lower()  # Convert to lowercase
    return text

# Testing the clean_text function on a sample text
sample_text = "   This is a \n\n\t sample    text.    "
print("Original text:", sample_text)
print("Cleaned text:", clean_text(sample_text))

Original text:    This is a 

	 sample    text.    
Cleaned text:    this is a 

	 sample    text.    


In [5]:
# Load and process the SQuAD dataset using the clean_text function
import json

def load_and_process_squad(filepath, max_samples=20000):
    """
    Loads, cleans, and extracts answerable questions from the SQuAD dataset.

    Args:
        filepath (str): Path to the SQuAD JSON file.
        max_samples (int): Maximum number of answerable questions to load.

    Returns:
        List[dict]: A list of cleaned question-answer pairs.
    """
    with open(filepath, "r") as f:
        squad_data = json.load(f)

    data = []
    for article in squad_data["data"]:
        for paragraph in article["paragraphs"]:
            context = clean_text(paragraph["context"]) 
            for qa in paragraph["qas"]:
                if qa["is_impossible"]:  
                    continue
                
                question = clean_text(qa["question"], is_question=True) 
                answer_text = clean_text(qa["answers"][0]["text"])  
                answer_start = qa["answers"][0]["answer_start"]
                
                data.append({
                    "context": context, 
                    "question": question, 
                    "answer": answer_text, 
                    "answer_start": answer_start, 
                    "answer_end": answer_start + len(answer_text)})


    data.sort(key=lambda x: (len(x["context"]), len(x["question"])))
    # Limit the number of samples if specified
    if max_samples > 0:
        data = data[:max_samples]
    return data

# Load and process the SQuAD dataset
train_dataset = load_and_process_squad("data/m2_train.json", max_samples=-1)
dev_samples = load_and_process_squad("data/m2_dev.json", max_samples=2000)

print("Number of QA pairs in train data:", len(train_dataset))
print("Training sample:", train_dataset[0].keys())
print("\n")
print("Number of QA pairs in test data:", len(dev_samples))
print("Dev sample:", dev_samples[0].keys())


Number of QA pairs in train data: 86821
Training sample: dict_keys(['context', 'question', 'answer', 'answer_start', 'answer_end'])


Number of QA pairs in test data: 2000
Dev sample: dict_keys(['context', 'question', 'answer', 'answer_start', 'answer_end'])


### 3. Tokenization

In [6]:
unique_words = set()

for sample in train_dataset:
    context = sample["context"]
    question = sample["question"]
    answer = sample["answer"]

    unique_words.update(set(re.findall(r"\w+", context)))
    unique_words.update(set(re.findall(r"\w+", question)))
    unique_words.update(set(re.findall(r"\w+", answer)))

                
# Print the number of unique words
print("Number of unique words in the training set:", len(unique_words))

Number of unique words in the training set: 82597


In [7]:
combined_text = []
for sample in train_dataset:
    combined_text.append(sample["context"])
    combined_text.append(sample["question"])
    combined_text.append(sample["answer"])

# Join all text samples into one corpus (you can also use '\n'.join for a more distinct separation)
combined_text = "\n".join(combined_text)


In [8]:
# Training the tokenizer
from scripts_m2 import *

# Creating / Loading the tokenizer
tokenizer = BPETokenizer()

# Training the tokenizer if not already trained
tokenizer.train(combined_text=combined_text.split("/n"), vocab_size=10000)


Loading tokenizer from ./tokenizers/tokenizer.json...
Tokenizer already exists at ./tokenizers/tokenizer.json. Skipping training.


In [9]:
# Encoding a Question
example_question = train_dataset[4]["question"]
tokenized_output, attention_mask = tokenizer.encode(example_question)
print("Tokenized Question:", tokenized_output)
print("Length of Tokenized Question:", len(tokenized_output))
print("Attention Mask:", attention_mask)

# Decoding the tokenized output
decoded_output = tokenizer.decode(tokenized_output)
print("Decoded Question:", decoded_output)

Tokenized Question: [3, 2441, 1997, 1966, 4721, 2776, 1974, 3539, 1978, 7873, 3120, 1390, 4]
Length of Tokenized Question: 13
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Decoded Question: where is the oklahoma school of science and mathematics located ?


In [10]:
# Changing the max tokenizer length
tokenizer.set_max_length(25)

In [11]:
# Encoding an answer
example_answer = train_dataset[4]["answer"]
tokenized_answer, attention_mask = tokenizer.encode(example_answer)
print("Tokenized Answer:", tokenized_answer)
print("Length of Tokenized Answer:", len(tokenized_answer))
print("Attention Mask:", attention_mask)

# Decoding the tokenized answer
decoded_answer = tokenizer.decode(tokenized_answer)
print("Decoded Answer:", decoded_answer)

Tokenized Answer: [3, 4721, 2251, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Length of Tokenized Answer: 25
Attention Mask: [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded Answer: oklahoma city


### 4. Dataset & Data Loaders

In [12]:
# Getting the dataset
train_dataset = load_and_process_squad("data/m2_train.json", max_samples=20000)
dev_dataset = load_and_process_squad("data/m2_dev.json", max_samples=2000)
print("Number of training samples:", len(train_dataset))
print("Number of dev samples:", len(dev_dataset))

Number of training samples: 20000
Number of dev samples: 2000


In [13]:
# Create the tokenizer
tokenizer = BPETokenizer()
# Train the tokenizer
tokenizer.train(combined_text=combined_text.split("\n"), vocab_size=10000)

Loading tokenizer from ./tokenizers/tokenizer.json...
Tokenizer already exists at ./tokenizers/tokenizer.json. Skipping training.


In [14]:
# View the training data
random_idx = np.random.randint(0, len(train_dataset))

# Print the random sample
print(train_dataset[random_idx].keys())
print("Training data sample:")
print(train_dataset[random_idx]["context"])
print(train_dataset[random_idx]["question"])
print(train_dataset[random_idx]["answer"])
print("Answer start:", train_dataset[random_idx]["answer_start"])
print("Answer end:", train_dataset[random_idx]["answer_end"])
print("Length of the training examples:", len(train_dataset))

dict_keys(['context', 'question', 'answer', 'answer_start', 'answer_end'])
Training data sample:
in 1913, his father was elevated to the nobility for his service to the austro-hungarian empire by emperor franz joseph. the neumann family thus acquired the hereditary appellation margittai, meaning of marghita. the family had no connection with the town; the appellation was chosen in reference to margaret, as was those chosen coat of arms depicting three marguerites. neumann jános became margittai neumann jános (john neumann of marghita), which he later changed to the german johann von neumann.
what town did von neumann's family become associated when elevated to nobility?
marghita
Answer start: 203
Answer end: 211
Length of the training examples: 20000


In [15]:
q = tokenizer.encode(train_dataset[4]["question"])[0]
tokenizer.decode(q)

'where is the oklahoma school of science and mathematics located ?'

In [16]:
# Viewing the length of the longest question, context, and answer
max_context_length = max(len(tokenizer.encode(sample["context"])[0]) for sample in train_dataset)
max_question_length = max(len(tokenizer.encode(sample["question"])[0]) for sample in train_dataset)
max_answer_length = max(len(tokenizer.encode(sample["answer"])[0]) for sample in train_dataset)

print("Max context length:", max_context_length)
print("Max question length:", max_question_length)
print("Max answer length:", max_answer_length)

Max context length: 205
Max question length: 64
Max answer length: 75


So for the transformer based models we can use the max length of the question and answer as follows:
1. `Answers Max Length`: 25 tokens
2. `Question Max Length`: 30 tokens
3. `Context Max Length`: 160 tokens
4. `Context + Question Length`: 160+30+1 (+1 for [SEP]) We could have used 175 as it is the 99th percentile but we will use 191 instead


This would give us a good tradeoff between the length of the sequence without using a lot of padding.

In [47]:
# Creating dataset
tokenizer = BPETokenizer()
train_data = load_and_process_squad("data/m2_train.json", max_samples=20000)
dev_data = load_and_process_squad("data/m2_dev.json", max_samples=2000)

context_max_length = 160
question_max_length = 30
answer_max_length = 25
train_dataset = QADataset(train_data, tokenizer, context_max_length=context_max_length, question_max_length=question_max_length, answer_max_length=answer_max_length, include_context=True, encode_two_texts_sep=True, context_question_swap=True)
dev_dataset = QADataset(dev_data, tokenizer, context_max_length=context_max_length, question_max_length=question_max_length, answer_max_length=answer_max_length, include_context=True, encode_two_texts_sep=True, context_question_swap=True)
print("Number of training samples:", len(train_dataset))
print("Number of dev samples:", len(dev_dataset))

# View a sample from the dataset
random_idx = np.random.randint(0, len(train_dataset))
print("Sample from the dataset:")
print("Question encoded:", train_dataset[random_idx]["question"])
print("Question decoded: ", tokenizer.decode(train_dataset[random_idx]["question"].tolist()))
print("Answer encoded:", train_dataset[random_idx]["answer"])
print("Answer decoded: ", tokenizer.decode(train_dataset[random_idx]["answer"].tolist()))
print("Context encoded:", train_dataset[random_idx]["context"])
print("Context decoded: ", tokenizer.decode(train_dataset[random_idx]["context"].tolist()))
print("Context Question encoded:", train_dataset[random_idx]["context_question"])
print("Context Question decoded: ", tokenizer.decode(train_dataset[random_idx]["context_question"].tolist()))
print("Answer Start:", train_dataset[random_idx]["answer_start"])
print("Answer End:", train_dataset[random_idx]["answer_end"])
print("Answer Start: End decoded:", tokenizer.decode(train_dataset[random_idx]["context"][train_dataset[random_idx]["answer_start"]:train_dataset[random_idx]["answer_end"]+1].tolist()))

print("\n\n")

# View a sample from the dev dataset
random_idx = np.random.randint(0, len(dev_dataset))
print("Sample from the dev dataset:")
print("Context encoded:", dev_dataset[random_idx]["context"])
print("Context decoded: ", tokenizer.decode(dev_dataset[random_idx]["context"].tolist()))
print("Question encoded:", dev_dataset[random_idx]["question"])
print("Question decoded: ", tokenizer.decode(dev_dataset[random_idx]["question"].tolist()))
print("Answer encoded:", dev_dataset[random_idx]["answer"])
print("Answer decoded: ", tokenizer.decode(dev_dataset[random_idx]["answer"].tolist()))
print("Context Question encoded:", dev_dataset[random_idx]["context_question"])
print("Context Question decoded: ", tokenizer.decode(dev_dataset[random_idx]["context_question"].tolist()))
print("Answer Start:", dev_dataset[random_idx]["answer_start"])
print("Answer End:", dev_dataset[random_idx]["answer_end"])
print("Answer Start: End decoded:", tokenizer.decode(dev_dataset[random_idx]["context"][dev_dataset[random_idx]["answer_start"]:dev_dataset[random_idx]["answer_end"]+1].tolist()))


Loading tokenizer from ./tokenizers/tokenizer.json...
Filtered dataset size: 19993 out of original 20000
Filtered dataset size: 1996 out of original 2000
Number of training samples: 19993
Number of dev samples: 1996
Sample from the dataset:
Question encoded: tensor([   3, 2087, 1997, 3564, 1967, 4828, 1390,    4,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1])
Question decoded:  what is okanye ?
Answer encoded: tensor([   3, 5982, 2071, 2041, 2014, 2748, 6615, 1315, 1282, 1315, 9702,    4,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1])
Answer decoded:  pronounce unstressed / o / clearly
Context encoded: tensor([   3, 1966, 3368, 3618, 5685, 1978, 2881, 5213, 3000, 1966, 2670, 5730,
        3267, 4162, 5982, 2071, 2041, 2014, 2748, 6615, 1315, 1282, 1315, 9702,
        1528, 1966, 8675, 1980, 2566, 3564, 1967, 4828, 1315,  303,  299,  289,


In [48]:
# Creating Dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True) # add this `num_workers=0` if you want to see print in __getitem__ in dataset class
dev_dataloader = DataLoader(dev_dataset, batch_size=32, shuffle=False)

for batch in train_dataloader:
    print(batch.keys())
    print(batch['context_question'].shape)
    print(batch['question'].shape)
    print(batch['answer'].shape)
    print(batch['attention_mask_question'].shape)
    print(batch['attention_mask_answer'].shape)
    print(batch['attention_mask_context_question'].shape)
    print(batch['answer_start'], batch['answer_end'])
    print(batch['context_question_type_mask'].shape)
    print("-" * 50)
    input_ids = batch['context_question']
    attention_mask = batch['attention_mask_context_question']
    token_type_ids = batch['context_question_type_mask']
    break

dict_keys(['question', 'attention_mask_question', 'answer', 'attention_mask_answer', 'context', 'attention_mask_context', 'context_question', 'attention_mask_context_question', 'answer_start', 'answer_end', 'context_question_type_mask'])
torch.Size([64, 189])
torch.Size([64, 30])
torch.Size([64, 25])
torch.Size([64, 30])
torch.Size([64, 25])
torch.Size([64, 189])
tensor([ 81,  19,   6,  76,  41, 123,  65,   0,  73,  40,  85,  78,  52,  71,
         39,  34,   5,  25,  24,  87,  22,   3,  14, 111, 120,  47,  49,  27,
        112,  27,  65,  12,  65, 101,  46,  23,   6,  68,  61,  58,  69,  19,
         60,  81,   0,  80,  93,  37,  12,  10,  23,  29,  49, 121,   9,  31,
         48,  28,  84,  15,   9,  45,  60,  56]) tensor([ 86,  25,  10,  76,  45, 123,  73,  13,  73,  40,  88,  81,  55,  71,
         44,  38,   7,  27,  28, 115,  23,   6,  15, 111, 121,  50,  51,  30,
        121,  51,  68,  17,  66, 102,  50,  23,   6,  87,  62,  62,  72,  21,
         67,  88,   2,  81,  99,  43,  

In [49]:
context_max_length + question_max_length

190

### 5. Phase 1: Model Training

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerQAModel(nn.Module):
    """
    Transformer Encoder QA model for SQuAD-style span prediction.
    Input:  [batch_size, seq_len] token IDs in the form
            <start> question <sep> context
    Output: start_logits, end_logits of shape [batch_size, seq_len]
    """
    def __init__(
        self,
        vocab_size: int,
        d_model: int = 300,
        nhead: int = 6,
        dim_feedforward: int = 2048,
        num_layers: int = 4,
        max_len: int = 512,
        dropout: float = 0.01,
        pad_idx: int = 0,
    ):
        super().__init__()
        # word + positional embeddings
        self.token_emb = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_emb   = nn.Embedding(max_len, d_model)

        # transformer encoder stack
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation="relu",
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # two linear heads for start/end prediction
        self.start_classifier = nn.Linear(d_model, 1)
        self.end_classifier   = nn.Linear(d_model, 1)

        self.dropout = nn.Dropout(dropout)
        self.max_len = max_len

    def forward(
        self,
        context_question: torch.LongTensor,               # [B, L]
        attention_mask_context_question: torch.BoolTensor = None,    # [B, L] (True = keep, False = pad)
    ):
        B, L = context_question.size()
        # create position IDs
        pos_ids = torch.arange(L, device=context_question.device).unsqueeze(0).expand(B, -1)

        # embed tokens + positions
        x = self.token_emb(context_question) + self.pos_emb(pos_ids)
        x = self.dropout(x)             # [B, L, D]

        # TransformerEncoder expects [L, B, D]
        x = x.transpose(0, 1)           # [L, B, D]

        # key_padding_mask: True for positions that should be ignored (pads)
        if attention_mask is not None:
            key_padding_mask = ~attention_mask_context_question.bool()  # invert: True=pad
        else:
            key_padding_mask = None

        # encode
        enc = self.encoder(x, src_key_padding_mask=key_padding_mask)
        enc = enc.transpose(0, 1)       # [B, L, D]

        # compute logits and squeeze
        start_logits = self.start_classifier(enc).squeeze(-1)  # [B, L]
        end_logits   = self.end_classifier(enc).squeeze(-1)    # [B, L]

        return start_logits, end_logits




In [ ]:

class TransformerQAModel3(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 d_model: int,
                 num_layers: int,
                 num_heads: int,
                 dim_feedforward: int,
                 max_question_len: int,
                 max_context_len: int,
                 dropout: float = 0.1):
        """
        Args:
          vocab_size: size of your tokenizer vocabulary
          d_model: transformer hidden size
          num_layers: number of encoder blocks
          num_heads: number of attention heads
          dim_feedforward: inner dim of the transformer FFN
          max_question_len: maximum length of question (m)
          max_context_len: maximum length of context (n)
        """
        super().__init__()
        self.d_model = d_model
        self.max_context_len = max_context_len
        self.seq_len = max_question_len + max_context_len - 1

        # --- embedding + positional encoding ---
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb   = nn.Embedding(self.seq_len, d_model)

        # --- transformer encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation="relu"
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # --- the “Concat & FeedForward” from Fig 2(a) ---
        #  input_dim = (context_len+1) * d_model, hidden = (context_len+1)
        self.fc1 = nn.LazyLinear(self.max_context_len)
        self.relu = nn.ReLU()

        # --- two heads for start / end ---
        self.start_ff = nn.LazyLinear(self.max_context_len)
        self.end_ff   = nn.LazyLinear(self.max_context_len)

    def forward(self,
                context_question: torch.LongTensor,
                attention_mask_context_question: torch.BoolTensor,
               ) -> torch.Tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
          context_question: LongTensor of shape (batch, seq_len)
          attention_mask_context_question: BoolTensor of shape (batch, seq_len),
                          True for real tokens, False for padding.
        Returns:
          start_probs, end_probs: both FloatTensors of shape (batch, context_len+1)
        """
        bsz, seq_len = context_question.size()
        assert seq_len == self.seq_len, \
            f"expected seq_len={self.seq_len}, got {seq_len}"

        # --- embed tokens + positions ---
        pos = torch.arange(seq_len, device=context_question.device).unsqueeze(0)  # (1, seq_len)
        x = self.token_emb(context_question) * math.sqrt(self.d_model)
        x = x + self.pos_emb(pos)         # (batch, seq_len, d_model)

        # --- transformer wants (seq_len, batch, d_model) ---
        x = x.transpose(0,1)

        # build src_key_padding_mask for transformer
        # (batch, seq_len), True = mask out; so invert attention_mask
        kp_mask = None
        if attention_mask_context_question is not None:
            kp_mask = attention_mask_context_question == 0  # (batch, seq_len)

        # --- pass through N transformer encoder layers ---
        x = self.encoder(x, src_key_padding_mask=kp_mask)  # (seq_len, batch, d_model)
        x = x.transpose(0,1)                               # (batch, seq_len, d_model)

        # --- slice off only the context portion (including [OOV]) ---
        # context lives at the *end* of the sequence:
        context_repr = x[:, -self.max_context_len:, :]   # (batch, C+1, d_model)

        # --- flatten and feed through the “Concat & FeedForward” ---
        flat = context_repr.contiguous().view(bsz, -1)     # (batch, (C+1)*d_model)
        h = self.relu(self.fc1(flat))                   # (batch, C+1)

        # --- two linear heads  ---
        start_logits = self.start_ff(h)                    # (batch, C+1)
        end_logits = self.end_ff(h)                      # (batch, C+1)
        # print("Start logits shape:", start_logits.shape)
        # print("End logits shape:", end_logits.shape)

        # attention_mask_context = attention_mask_context_question[:, -self.max_context_len:]
        # # print("Attention mask context shape:", attention_mask_context.shape)
        # ctx_pad = attention_mask_context == 0 
        # start_logits = start_logits.masked_fill(ctx_pad, -1e9) # (batch, C+1)
        # end_logits = end_logits.masked_fill(ctx_pad, -1e9) # (batch, C+1)

        print("argmax start logits:", start_logits.argmax(dim=-1))
        print("argmax end logits:", end_logits.argmax(dim=-1))

        return start_logits, end_logits

In [59]:
import torch
import torch.nn as nn
import math

class TransformerQAModel3(nn.Module):
    def __init__(self,
                 vocab_size: int,
                 d_model: int,
                 num_layers: int,
                 num_heads: int,
                 dim_feedforward: int,
                 max_question_len: int,
                 max_context_len: int,
                 dropout: float = 0.1):
        """
        Args:
          vocab_size: size of your tokenizer vocabulary
          d_model: transformer hidden size
          num_layers: number of encoder blocks
          num_heads: number of attention heads
          dim_feedforward: inner dim of the transformer FFN
          max_question_len: maximum length of question (m)
          max_context_len: maximum length of context (n)
        """
        super().__init__()
        self.d_model = d_model
        self.max_context_len = max_context_len
        self.seq_len = max_question_len + max_context_len - 1

        # --- Embedding + Positional encoding ---
        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb   = nn.Embedding(self.seq_len, d_model)

        # --- Transformer encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=num_heads,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation="relu"
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # --- Dropout layer ---
        self.dropout = nn.Dropout(dropout)

        # --- Two linear heads for start / end (per token) ---
        self.start_ff = nn.Linear(d_model, 1)
        self.end_ff   = nn.Linear(d_model, 1)

    def forward(self,
                context_question: torch.LongTensor,
                attention_mask_context_question: torch.BoolTensor
               ) -> torch.Tuple[torch.Tensor, torch.Tensor]:
        """
        Args:
          context_question: LongTensor of shape (batch, seq_len)
          attention_mask_context_question: BoolTensor of shape (batch, seq_len)
        Returns:
          start_logits, end_logits: FloatTensors of shape (batch, context_len+1)
        """
        bsz, seq_len = context_question.size()
        assert seq_len == self.seq_len, \
            f"expected seq_len={self.seq_len}, got {seq_len}"

        # --- Embedding ---
        pos = torch.arange(seq_len, device=context_question.device).unsqueeze(0)  # (1, seq_len)
        x = self.token_emb(context_question) * math.sqrt(self.d_model)
        x = x + self.pos_emb(pos)         # (batch, seq_len, d_model)

        x = self.dropout(x)
        x = x.transpose(0, 1)             # (seq_len, batch, d_model)

        # Invert attention mask for transformer: True = padding
        kp_mask = attention_mask_context_question == 0

        # --- Pass through transformer encoder ---
        x = self.encoder(x, src_key_padding_mask=kp_mask)  # (seq_len, batch, d_model)
        x = x.transpose(0, 1)                              # (batch, seq_len, d_model)

        # --- Extract context representations (including [OOV]) ---
        context_repr = x[:, -self.max_context_len:, :]     # (batch, C+1, d_model)
        context_repr = self.dropout(context_repr)

        # --- Token-level start/end logits ---
        start_logits = self.start_ff(context_repr).squeeze(-1)  # (batch, C+1)
        end_logits   = self.end_ff(context_repr).squeeze(-1)    # (batch, C+1)

        print("argmax start logits shape:", start_logits.argmax(dim=-1))
        print("argmax end logits shape:", end_logits.argmax(dim=-1))

        return start_logits, end_logits

In [60]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pad_idx=1

model = model = TransformerQAModel3(
    vocab_size=10_000,
    d_model=256,
    num_layers=6,
    num_heads=8,
    dim_feedforward=512,
    max_question_len=30,
    max_context_len=160,
    dropout=0
)
model = model.to(device)

# Setup an optimizer (e.g., Adam)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

train_qa_context_model_boilerplate(
    model=model, 
    train_dataloader=train_dataloader, 
    val_dataloader=dev_dataloader, 
    optimizer=optimizer, 
    criterion=criterion, 
    num_epochs=10, 
    device=device, 
    inputs = ["context_question", "attention_mask_context_question"],
    evaluate_val_dataset=True,
)


c:\Users\001\anaconda3\envs\nlp_env\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/10:   0%|          | 0/313 [00:00<?, ?it/s]

argmax start logits shape: 

Epoch 1/10:   0%|          | 1/313 [00:00<02:05,  2.49it/s, loss=9.27]

tensor([  6,  54,  16,  30,  91,  95,  17,  68,  71,   2,  55, 107, 123,  78,
         25,  66,  19,  97,  71,  56,  34,  27,  89,  55, 100,  20,  65,   7,
         42,  94,   9,  33,  40,  18,  35,  53,  37,  33,  48,  60,  75,  87,
         36,  43,  11,  41,  77,  90,  45,  31,   6,  78,  34,  24,  69,  41,
         73,  51,  35, 114, 105,  23,  43,  28], device='cuda:0')
argmax end logits shape: tensor([ 69, 110,  78, 123, 139, 112,  12,   8, 137,  33,  98, 120, 129,  84,
        115,  82,  36,  78,  56, 134,  38, 109, 115,  90, 121,  89, 123,  66,
        103,  94,  60,  10,  74,  54,  93,  89,  75,  17,  95, 115, 112,  94,
         99, 145,  96, 109,  55, 159, 110, 126,  84,  42,  78, 104, 108, 109,
         76,  52,   8, 127,  69, 110, 137,  30], device='cuda:0')
argmax start logits shape: tensor([ 20,  42,  28,  99,  30,  71,  58,  36,   2,  16,  81,  81,  15,  24,
         92,   3,   9,  79,  65,  79,  38,  96,  11,  62,  43,  14,  30,  47,
         98,  12,  37, 116,  27,  28

Epoch 1/10:   2%|▏         | 5/313 [00:00<00:34,  9.05it/s, loss=8.91]

...

Evaluating:  51%|█████     | 160/313 [00:04<00:04, 32.75it/s]

argmax start logits shape: tensor([120,  25,  20,  33,   2,  58,  65,   3,  26, 108,   0,  14, 112,  93,
         34,  16,  62,  16,   0,  19,   3,  93,   2,  63,  70,  43,  92,  45,
          2,  83,  42,   2,   0,   0,   0,  69,  28,  87,  36,  81,  44,  80,
        103,  78,   0, 107,  74,   0,  83,   9,  26,  80,  53,  48,  53,   2,
         74,  83,  65,  26,  10,   0,  33,   2], device='cuda:0')
argmax end logits shape: tensor([ 49,  29,  12,  48,  54,  21, 108,  78,  32,  26,  80,  95,  79,  49,
         14,  13,   3,   5,  59,  38,   6,  13,  30,  63,  20,  17,  10,  12,
         40,  43, 130,   2,  79,  23,  23,  72,  56,  78,  32,  93,  63,  15,
        103,  84,  10,  16,  13,  14,  73,  32,  16,  80,  40, 100, 102,   5,
         62,   3,  37,  15, 125,  24,   2,  68], device='cuda:0')
argmax start logits shape: tensor([ 21,  50,   5, 136,  46,  50,   3,  63,   0,  63,  47,   0,  73,  40,
         29,  16,  16,  19,  55,  67,   1,  33,   0,   9, 107,  55,  39,  38,
         

Evaluating:  52%|█████▏    | 162/313 [00:04<00:04, 33.42it/s]

argmax start logits shape: tensor([ 61,   2,  75,  37,   4,  45,   0,   5,  80,  71,  16,  54,   3,  14,
         51,  19,  78,  38, 120,   0,   4,   0,  48,  32, 117,  33,  20,   9,
         26, 127,   0,  88, 101,  18,  52,  69,  58,   0, 127,  25,  72, 100,
         38,   0,  22,  55,  33,  29,  65,  30,  95,   5,  24, 127,  13,  51,
          0,  62,  41,   0,  11,   8,   0,  43], device='cuda:0')
argmax end logits shape: tensor([ 64,  61,  37,  38,  45,  47,  69,  86,  35,  45,   8,  54,  22,  14,
         17,  29, 105,  39,  88,  55,  30,  48,  48,   4,  48,  45, 112,  82,
         51,  17,  12,  88,  21, 109, 141,  41,  58,  37,  39, 112,  77, 101,
         39,  11,  54,  78,  12,  85,  49,   6,  96,  73,   4,  58,  17,  90,
         19,  67,  32,  90,  58,   2,  67,  17], device='cuda:0')
argmax start logits shape: 

KeyboardInterrupt: 

In [40]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pad_idx=1

model = TransformerQAModel(vocab_size=10000)
model = model.to(device)

# Setup an optimizer (e.g., Adam)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

train_qa_context_model_boilerplate(
    model=model, 
    train_dataloader=train_dataloader, 
    val_dataloader=dev_dataloader, 
    optimizer=optimizer, 
    criterion=criterion, 
    num_epochs=10, 
    device=device, 
    inputs = ["context_question", "attention_mask_context_question"],
    evaluate_val_dataset=True,
)


c:\Users\001\anaconda3\envs\nlp_env\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Epoch 1/10: 100%|██████████| 625/625 [00:28<00:00, 21.89it/s, loss=9.7] 


Epoch 1 Loss: 9.6254


Evaluating: 100%|██████████| 625/625 [00:11<00:00, 54.76it/s]


Training Loss: 9.4978
Training Metrics: {'start_accuracy': 0.03336167658680538, 'start_precision': 0.0025512420609037755, 'start_recall': 0.03336167658680538, 'start_f1_score': 0.0041695202896415914, 'end_accuracy': 0.013354674135947582, 'end_precision': 0.002648366725810521, 'end_recall': 0.013354674135947582, 'end_f1_score': 0.0017788785126064577, 'joint_exact_match': 0.004951733106587306, 'span_overlap_f1': 0.04972476770789385}


Evaluating: 100%|██████████| 63/63 [00:01<00:00, 55.28it/s]


...

Validation Loss: 10.4835
Validation Metrics: {'start_accuracy': 0.009018036072144289, 'start_precision': 0.0072049700008965534, 'start_recall': 0.009018036072144289, 'start_f1_score': 0.0054157054795576755, 'end_accuracy': 0.008016032064128256, 'end_precision': 0.003922685675360828, 'end_recall': 0.008016032064128256, 'end_f1_score': 0.004407336472507647, 'joint_exact_match': 0.002004008016032064, 'span_overlap_f1': 0.01473049934691032}
--------------------------------------------------


Epoch 4/10: 100%|██████████| 625/625 [00:28<00:00, 22.03it/s, loss=10.5]


Epoch 4 Loss: 10.4859


Evaluating:  70%|███████   | 440/625 [00:08<00:03, 53.93it/s]


KeyboardInterrupt: 

In [64]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


class PositionalEncoding(nn.Module):
    """
    Adds sinusoidal positional encodings to token embeddings.
    """
    def __init__(self, d_model, dropout=0.2, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)

    def forward(self, x):
        # x: (batch, seq_len, d_model)
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


class BiDAFAttention(nn.Module):
    """
    Bi-Directional Attention Flow layer from "BiDAF" (Seo et al., 2017).
    """
    def __init__(self, hidden_dim):
        super(BiDAFAttention, self).__init__()
        # similarity takes [c, q, c ∘ q]
        self.similarity_linear = nn.Linear(3 * hidden_dim, 1, bias=False)

    def forward(self, context, query):
        batch_size, c_len, _ = context.size()
        q_len = query.size(1)

        # pairwise combinations
        c_exp = context.unsqueeze(2).expand(-1, -1, q_len, -1)
        q_exp = query.unsqueeze(1).expand(-1, c_len, -1, -1)
        cq_mul = c_exp * q_exp
        sim_in = torch.cat([c_exp, q_exp, cq_mul], dim=3)

        # similarity matrix
        S = self.similarity_linear(sim_in).squeeze(3)  # (B, c_len, q_len)

        # Context-to-Query
        a = F.softmax(S, dim=2)                       # (B, c_len, q_len)
        c2q = torch.bmm(a, query)                     # (B, c_len, D)

        # Query-to-Context
        b = F.softmax(torch.max(S, dim=2)[0], dim=1)  # (B, c_len)
        b = b.unsqueeze(1)                            # (B, 1, c_len)
        q2c = torch.bmm(b, context)                   # (B, 1, D)
        q2c = q2c.repeat(1, c_len, 1)                  # (B, c_len, D)

        # combine
        G = torch.cat([context, c2q, context * c2q, context * q2c], dim=2)
        return G


class BiDAFOutput(nn.Module):
    """
    Output layer to compute start and end logits for BiDAF/G
    """
    def __init__(self, hidden_size, dropout=0.2):
        super(BiDAFOutput, self).__init__()
        # G dim = 8*H, M dim = 2*H -> concat dim = 10*H
        self.p1_weight = nn.Linear(10 * hidden_size, 1)
        self.p2_weight = nn.Linear(10 * hidden_size, 1)
        self.modelling_LSTM_end = nn.LSTM(
            2 * hidden_size, hidden_size, bidirectional=True, batch_first=True
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, G, M):
        # G: (B, c_len, 8H), M: (B, c_len, 2H)
        GM = torch.cat([G, M], dim=2)  # (B, c_len, 10H)
        start_logits = self.p1_weight(self.dropout(GM)).squeeze(2)

        M2, _ = self.modelling_LSTM_end(M)
        GM2 = torch.cat([G, M2], dim=2)
        end_logits = self.p2_weight(self.dropout(GM2)).squeeze(2)
        return start_logits, end_logits


class BiDAFTransformer(nn.Module):
    """
    BiDAF with Transformers replacing the BiLSTMs.
    """
    def __init__(
        self,
        vocab_size: int,
        embed_dim: int,
        hidden_size: int,
        nhead: int = 8,
        num_layers: int = 2,
        dim_feedforward: int = 2048,
        dropout: float = 0.2,
        pretrained_embeddings: torch.Tensor = None
    ):
        super().__init__()
        self.hidden_dim = 2 * hidden_size  # match BiLSTM output

        # embeddings + projection
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
        self.embed_proj = nn.Linear(embed_dim, self.hidden_dim)
        self.pos_enc = PositionalEncoding(self.hidden_dim, dropout)

        # transformer encoder for context & question
        enc_layer = TransformerEncoderLayer(
            d_model=self.hidden_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.context_encoder = TransformerEncoder(enc_layer, num_layers=num_layers)

        # attention flow
        self.att_flow = BiDAFAttention(self.hidden_dim)

        # modeling transformer
        self.modeling_proj = nn.Linear(4 * self.hidden_dim, self.hidden_dim)
        mod_layer = TransformerEncoderLayer(
            d_model=self.hidden_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            activation='relu'
        )
        self.modeling_encoder = TransformerEncoder(mod_layer, num_layers=num_layers)

        # output layer
        self.output_layer = BiDAFOutput(hidden_size, dropout)

    def forward(self, context, question, attention_mask_context, attention_mask_question):
        ctx_mask = attention_mask_context == 0
        qry_mask = attention_mask_question == 0
        

        # embed + project + pos encode
        c = self.embedding(context)
        c = self.embed_proj(c)
        c = self.pos_enc(c)
        q = self.embedding(question)
        q = self.embed_proj(q)
        q = self.pos_enc(q)

        # transformer encoders (seq_len, batch, dim)
        c_enc = self.context_encoder(
            c.transpose(0,1), src_key_padding_mask=ctx_mask
        ).transpose(0,1)
        q_enc = self.context_encoder(
            q.transpose(0,1), src_key_padding_mask=qry_mask
        ).transpose(0,1)

        # attention flow
        G = self.att_flow(c_enc, q_enc)

        # modeling
        M_in = self.modeling_proj(G)
        M = self.modeling_encoder(
            M_in.transpose(0,1), src_key_padding_mask=ctx_mask
        ).transpose(0,1)

        # output heads
        start_logits, end_logits = self.output_layer(G, M)
        return start_logits, end_logits


In [65]:
import torch

# 1) Set up model hyperparameters
vocab_size    = 10_000    # e.g. BERT‐style vocab
embed_dim     = 300      # your embedding size
hidden_size   = 128      # LSTM “half”–size; transformer d_model = 2*hidden_size

# 2) Instantiate the model
model = BiDAFTransformer(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    nhead=8,
    num_layers=2,
    dim_feedforward=512,
    dropout=0.2,
    pretrained_embeddings=None  # or your Tensor(shape=(vocab_size, embed_dim))
)

model = model.to(device)

# Setup an optimizer (e.g., Adam)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

train_qa_context_model_boilerplate(
    model=model, 
    train_dataloader=train_dataloader, 
    val_dataloader=dev_dataloader, 
    optimizer=optimizer, 
    criterion=criterion, 
    num_epochs=10, 
    device=device, 
    inputs = ["context", "question", "attention_mask_context", "attention_mask_question"],
    evaluate_val_dataset=True,
)

Epoch 1/10: 100%|██████████| 313/313 [00:25<00:00, 12.32it/s, loss=8.61]


Epoch 1 Loss: 8.4773


Evaluating: 100%|██████████| 313/313 [00:10<00:00, 30.60it/s]


Training Loss: 8.4181
Training Metrics: {'start_accuracy': 0.05016755864552593, 'start_precision': 0.050738859724243336, 'start_recall': 0.05016755864552593, 'start_f1_score': 0.04446677830471992, 'end_accuracy': 0.06667333566748362, 'end_precision': 0.0620882369875789, 'end_recall': 0.06667333566748362, 'end_f1_score': 0.061667272291193446, 'joint_exact_match': 0.004251488020807282, 'span_overlap_f1': 0.058614727536063435}


Evaluating: 100%|██████████| 63/63 [00:01<00:00, 58.21it/s]


...

Validation Loss: 9.0228
Validation Metrics: {'start_accuracy': 0.022545090180360723, 'start_precision': 0.022433151114667454, 'start_recall': 0.022545090180360723, 'start_f1_score': 0.02056413705847162, 'end_accuracy': 0.036072144288577156, 'end_precision': 0.03463007143764952, 'end_recall': 0.036072144288577156, 'end_f1_score': 0.03156325627422017, 'joint_exact_match': 0.009519038076152305, 'span_overlap_f1': 0.03403111395304026}
--------------------------------------------------


Epoch 7/10: 100%|██████████| 313/313 [00:24<00:00, 12.94it/s, loss=8.1] 


Epoch 7 Loss: 8.2732


Evaluating:   0%|          | 0/313 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
model_path = "models/qa_context_model_trans_cross_att.pkl"

In [33]:
save_model(model, model_path)

Model saved to models/qa_context_model_dqra.pkl


In [34]:
model = load_model(model_path=model_path)

Model loaded from models/qa_context_model_dqra.pkl


In [41]:
# evaluate the model on the dev set
criterion = nn.CrossEntropyLoss(ignore_index=1)
evaluate_qa_context_model_boilerplate(model=model, dataloader=train_dataloader, criterion=criterion, device='cuda', inputs = ["context", "question", "attention_mask_question"])

Evaluating:   0%|          | 0/625 [00:00<?, ?it/s]


TypeError: TransformerQAModel.forward() got an unexpected keyword argument 'context'

In [22]:
# evaluate the model on the dev set
criterion = nn.CrossEntropyLoss(ignore_index=1)
evaluate_qa_context_model_boilerplate(model=model, dataloader=dev_dataloader, criterion=criterion, device='cuda', inputs = ["context", "question", "attention_mask_question"])

Evaluating: 100%|██████████| 63/63 [00:01<00:00, 50.07it/s]

Validation Loss: 10.3172
Validation Metrics: {'start_accuracy': 0.07565130260521043, 'start_precision': 0.09645009422754619, 'start_recall': 0.07565130260521043, 'start_f1_score': 0.07168606723489031, 'end_accuracy': 0.09018036072144289, 'end_precision': 0.13011087755691064, 'end_recall': 0.09018036072144289, 'end_f1_score': 0.08554795701482384, 'joint_exact_match': 0.02404809619238477, 'span_overlap_f1': 0.07026853459209209}


(10.317197020091708,
 {'start_accuracy': 0.07565130260521043,
  'start_precision': 0.09645009422754619,
  'start_recall': 0.07565130260521043,
  'start_f1_score': 0.07168606723489031,
  'end_accuracy': 0.09018036072144289,
  'end_precision': 0.13011087755691064,
  'end_recall': 0.09018036072144289,
  'end_f1_score': 0.08554795701482384,
  'joint_exact_match': 0.02404809619238477,
  'span_overlap_f1': 0.07026853459209209})

In [42]:
preds, true_labels = predict_qa_context_model_boilerplate(model=model, dataloader=dev_dataloader, tokenizer=tokenizer, device='cuda', inputs = ["context_question", "attention_mask_context_question"])

Predicting: 100%|██████████| 63/63 [00:01<00:00, 45.76it/s]


In [43]:
# Compare the predictions with the actual answers
for i in range(100):
    print(f"Predicted Answer: {preds[i]}")
    print(f"True Answer: {true_labels[i]}")
    print("-" * 50)

Predicted Answer: 
True Answer: rugby
--------------------------------------------------
Predicted Answer: ,
True Answer: an official school sport
--------------------------------------------------
Predicted Answer: 
True Answer: high school
--------------------------------------------------
Predicted Answer: the
True Answer: framework
--------------------------------------------------
Predicted Answer: 
True Answer: complexity classes
--------------------------------------------------
Predicted Answer: ,
True Answer: complicated definitions
--------------------------------------------------
Predicted Answer: and
True Answer: palm springs
--------------------------------------------------
Predicted Answer: for
True Answer: southern
--------------------------------------------------
Predicted Answer: be
True Answer: open spaces
--------------------------------------------------
Predicted Answer: its
True Answer: beaches
--------------------------------------------------
Predicted Answer

In [25]:
train_preds, train_true_labels = predict_qa_context_model_boilerplate(model=model, dataloader=train_dataloader, tokenizer=tokenizer, device='cuda', inputs = ["context", "question", "attention_mask_question"])

Predicting: 100%|██████████| 625/625 [00:14<00:00, 44.49it/s]


In [26]:
# Compare the predictions with the actual answers
for i in range(100):
    print(f"Predicted Answer: {train_preds[i]}")
    print(f"True Answer: {train_true_labels[i]}")
    print("-" * 50)

Predicted Answer: portuguese
True Answer: coimbra
--------------------------------------------------
Predicted Answer: fatburger
True Answer: fatburger
--------------------------------------------------
Predicted Answer: 
True Answer: rigid descent rule
--------------------------------------------------
Predicted Answer: upper case ( also capital letters , capitals , caps , large letters , or more formally majuscule , see terminology ) and smaller lower case ( also small letters , or more formally minuscule , see terminology ) in the written representation of certain languages . here is a comparison of the upper and lower case versions of each letter included in the english alphabet
True Answer: larger upper
--------------------------------------------------
Predicted Answer: 
True Answer: statistical update
--------------------------------------------------
Predicted Answer: 118 % from 2012 – 13 when coverage still aired on fox soccer and espn / espn2 ( 220 , 000 viewers ), and nbc sp